In [1]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface
!pip install chromadb llama-index-vector-stores-chroma
!pip install openai
!pip install gradio



In [2]:
import os
import warnings
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from huggingface_hub import notebook_login
from openai import OpenAI
import gradio as gr


In [3]:
try:
    notebook_login()
except Exception as e:
    print(f"Hugging Face login failed: {e}")


In [4]:
!wget "https://filedn.eu/l1MYFwJMIh4Y60BIIrYyMiy/Quran_Persian_QA/my_chroma_index%20.zip"



--2025-07-19 12:35:46--  https://filedn.eu/l1MYFwJMIh4Y60BIIrYyMiy/Quran_Persian_QA/my_chroma_index%20.zip
Resolving filedn.eu (filedn.eu)... 185.62.236.186
Connecting to filedn.eu (filedn.eu)|185.62.236.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 226878308 (216M) [application/zip]
Saving to: ‘my_chroma_index .zip.1’

my_chroma_index .zi 100%[===================>] 216.37M  39.4MB/s    in 6.2s    

2025-07-19 12:35:53 (35.0 MB/s) - ‘my_chroma_index .zip.1’ saved [226878308/226878308]



In [5]:
import zipfile

with zipfile.ZipFile('my_chroma_index .zip','r') as zip_ref:
  zip_ref.extractall('my_chroma_index')
print(os.listdir('my_chroma_index'))

['content']


In [6]:
PERSIST_DIR = "/content/my_chroma_index/content/my_chroma_index"
EMBED_MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
OPENROUTER_API_KEY = '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'


In [7]:
def load_existing_index():
    """Load the pre-created ChromaDB index"""
    try:
        print("Loading embedding model...")
        embed_model = HuggingFaceEmbedding(model_name=EMBED_MODEL_NAME)

        print("Connecting to ChromaDB...")
        chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

        # List all collections to see what's available
        collections = chroma_client.list_collections()
        print(f"Found {len(collections)} collections:")
        for col in collections:
            print(f"  - {col.name} (ID: {col.id})")

        if not collections:
            raise Exception("No collections found in the ChromaDB database")

        # Use the first collection (or try to find the right one)
        collection_name = collections[0].name
        print(f"Using collection: {collection_name}")

        chroma_collection = chroma_client.get_collection(name=collection_name)
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

        print("Loading vector index...")
        index = VectorStoreIndex.from_vector_store(
            vector_store=vector_store,
            embed_model=embed_model
        )
        print("Index loaded successfully!")
        return index

    except Exception as e:
        print(f"Error loading index: {e}")
        print("\nDebugging information:")
        print(f"Looking for ChromaDB at: {PERSIST_DIR}")

        if os.path.exists(PERSIST_DIR):
            print("✓ ChromaDB directory exists")
            print("Directory contents:")
            for root, dirs, files in os.walk(PERSIST_DIR):
                level = root.replace(PERSIST_DIR, '').count(os.sep)
                indent = ' ' * 2 * level
                print(f"{indent}{os.path.basename(root)}/")
                subindent = ' ' * 2 * (level + 1)
                for file in files:
                    print(f"{subindent}{file}")
        else:
            print("✗ ChromaDB directory not found")

        # Try to reinitialize ChromaDB if it's corrupted
        print("\nTrying to reinitialize ChromaDB...")
        try:
            chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)
            print("ChromaDB client created successfully")
        except Exception as reinit_error:
            print(f"Failed to reinitialize: {reinit_error}")

        raise



In [8]:
def answer_query(query, index, top_k=5):
    """Answer query using RAG with the loaded index"""
    try:
        # Retrieve relevant documents
        retriever = index.as_retriever(similarity_top_k=top_k)
        nodes = retriever.retrieve(query)
        context = "\n\n".join([node.text for node in nodes])

        # Create prompt
        prompt = f"""
تو یک دستیار فارسی زبان هستی که به سوالات بر اساس استناد به قرآن پاسخ جامع و کامل میدهی

اطلاعات:
{context}

سوال:
{query}

اگر سوال کاربر نامرتبط به دین و قران بود بگو که نمیتوانی پاسخ دهی. همچنین مرحله فکر کردنت را به کاربر نشان نده
        """

        # Call OpenRouter API
        from openai import OpenAI
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=OPENROUTER_API_KEY,
        )

        completion = client.chat.completions.create(
            model="deepseek/deepseek-chat-v3-0324:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=1500
        )

        return completion.choices[0].message.content

    except Exception as e:
        return f"خطا در پاسخ دادن: {e}"

# Load the pre-created index
print("Loading the pre-created index...")

index = load_existing_index()


Loading the pre-created index...
Loading embedding model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Connecting to ChromaDB...
Found 1 collections:
  - my_collection (ID: 7de4729f-5f9c-4477-8f3c-0d4e2d07b703)
Using collection: my_collection
Loading vector index...
Index loaded successfully!


In [9]:
def interactive_qa():
    """Interactive question-answering session"""
    print("\n" + "="*50)
    print("حالت تعاملی - برای خروج 'quit' تایپ کنید")
    print("="*50)

    while True:
        query = input("\nسوال شما: ")
        if query.lower() in ['quit', 'exit', 'خروج']:
            break

        print("پاسخ:")
        response = answer_query(query, index)
        print(response)

In [ ]:
print(interactive_qa())


حالت تعاملی - برای خروج 'quit' تایپ کنید

سوال شما: رمز موفقیت در زندگی چیست؟
پاسخ:
**پاسخ:**  
رمز موفقیت در زندگی از منظر قرآن، بر پایه اصولی است که خداوند در آیات خود بیان فرموده است. برخی از این اصول کلیدی عبارتند از:  

1. **ایمان و توکل بر خداوند**  
   - قرآن می‌فرماید:  
     **«وَ مَنْ یَتَوَکَّلْ عَلَی اللَّهِ فَهُوَ حَسْبُهُ»** (طلاق: ۳)  
     (هر کس بر خدا توکل کند، او برایش کافی است).  
   - توکل به خداوند و ایمان به قضا و قدر الهی، آرامش و جهت‌دهی صحیح به زندگی می‌بخشد.  

2. **عمل صالح و تلاش مستمر**  
   - قرآن کریم عمل نیک را همراه ایمان، شرط رسیدن به سعادت می‌داند:  
     **«مَنْ عَمِلَ صَالِحًا مِنْ ذَکَرٍ أَوْ أُنْثَیٰ وَ هُوَ مُؤْمِنٌ فَلَنُحْیِیَنَّهُ حَیَاهً طَیِّبَهً»** (نحل: ۹۷)  
     (هر کس عمل شایسته انجام دهد، مرد یا زن، در حالی که مؤمن است، به او زندگی پاکیزه‌ای خواهیم بخشید).  

3. **صبر و استقامت در مشکلات**  
   - قرآن صبر را کلید پیروزی می‌داند:  
     **«إِنَّمَا یُوَفَّی الصَّابِرُونَ أَجْرَهُمْ بِغَیْرِ حِسَابٍ»** (زمر: ۱۰)  
     (بی‌گمان صابران 